In [27]:
import pandas as pd
import math
import numpy as np

In [ ]:
# Definiujemy funkcje aktywacji fi (sigmoida)
# fi' = fi*(1-fi) 

def sigmoid(x): return (1+np.exp(-x))**(-1) 

def deriv_sigmoid(x): return sigmoid(x)*(1-sigmoid(x))

In [ ]:
# Generujemy wagi
def wygeneruj_wagi(wejscie, wyjscie):
    # wejsciem jest liczba neuronow w obecnej warstwie
    # wyjściem jest liczba neuronów w następnej warstwie
    wektor_wag = np.random.normal(0,1/math.sqrt(len(wejscie)),(1+len(wejscie))*wyjscie)
    return np.reshape(wektor_wag, (wyjscie, 1+len(wejscie) )  )

In [4]:
wygeneruj_wagi((1,2), 2)

array([[-0.35723577,  1.21747645, -1.22620051],
       [-1.25128079,  0.12543397,  0.34059919]])

In [5]:
# Wejscie = wektor 784
# Wyjscie = wektor 10

# nerunony_na_warstwe: podajemy liczbę na warstwy głębokie
def forward_prop(wejscie_dane, liczb_neuronow_wyjscie ):
    """Podajemy dane na wejście, dodajemy na bias = 1 na początek wektora (otrzymujemy w ten sposób X).
    Zadajemy najpierw ile chcemy mieć neuronów na wyjściu
    Nastepnie liczymy net = W*X <- wyjście netto.
    Póżniej nakładamy funkcję aktywacji fi na net, otrzymując fi(net) = a <- wyjście z warstwy.
    Musimy też zapisywać stany x, net, a dla każdej warstwy (np. lista krotek)"""

    stany_x_net_a = []
    # Musimy przerobić X na wektor kolumnowy
    X = np.vstack([1,wejscie_dane.reshape(-1,1)])
    W = wygeneruj_wagi(wejscie_dane, liczb_neuronow_wyjscie)
    net = W @ X
    a = sigmoid(net)
    stany_x_net_a.append((X,net,a))
    return stany_x_net_a


In [6]:
A1 = forward_prop(np.array([1,1,0]), liczb_neuronow_wyjscie=2)[0][2]
print(A1)

A2 = forward_prop(A1, liczb_neuronow_wyjscie=2)[0][2]
print(A2)

[[0.17980467]
 [0.30104507]]
[[0.75236525]
 [0.45737023]]


In [8]:
# Definiujemy propagację wstecz

# wyjscie oczekiwane - to co chcielibyśmy uzyskać
# wyjscie dane - nasz wynik z forward_prop

# Funkcja straty to L = 1/2(a[L]-y)^2, czyli pochodna z L to a[L]-y
def back_prop(wyjscie_oczekiwane, wyjscie_dane, wagi_warstwy, stala_uczenia = 0.01 ):

    # Ostatnia warstwa
   # a_ost = 
    dL_a_wyjscia = wyjscie_dane - wyjscie_oczekiwane

    # mamy pochodną dL/da * fi(net)
    delta = dL_a_wyjscia * deriv_sigmoid( inv_sigmoid(wyjscie_dane) )
    dL_dW = delta * wyjscie_dane.T

    # Liczymy dL/dX i usuwamy pierwszy element, otrzymując dL/da niższej warstwy
    dL_dX = wagi_warstwy.T @ delta

    dL_da_nizsze = dL_dX[1:]

    # Aktualizujemy wagi
    W_nowe = wagi_warstwy - stala_uczenia*dL_dW


In [9]:
A = np.array([[0.1,-0.2,0.3], [-0.4,0.5,-0.6]])
B = np.array([1,1,0])
C= B.reshape(-1,1)
C

array([[1],
       [1],
       [0]])

In [10]:
A@B

array([-0.1,  0.1])

In [11]:
np.hstack([1,A])

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
sigmoid(C)

TypeError: only length-1 arrays can be converted to Python scalars

In [32]:
liczba_neuronow = 3
liczba_wejsc = 4
x= np.random.randn(liczba_neuronow, liczba_wejsc + 1) * 1/np.sqrt(liczba_wejsc)
y = np.random.normal(0,1/math.sqrt(liczba_wejsc),(1+liczba_wejsc)*liczba_neuronow)

print(x)

[[ 0.39628975 -0.08171137 -0.00218662 -0.48415335 -0.11996831]
 [-0.34005076 -0.05461619 -0.3453193   0.02321433  0.56519641]
 [ 0.07350721  0.65815865  0.68618035  0.14726593  0.78687395]]


In [3]:
# Musimy unormować dane
def normuj_dane(dane):
    unormowane = []
    for  x, y in dane:
        x_norm = x/255
        unormowane.append((x_norm,y))
    return unormowane

In [33]:
import numpy as np
import math
import pickle
import gzip
from mnist_loader import load_data_wrapper

class Warstwa:
    def __init__(self, liczba_wejsc, liczba_neuronow, funkcja_aktywacji='sigmoid'):
        self.liczba_wejsc = liczba_wejsc
        self.liczba_neuronow = liczba_neuronow
        
        # Inicjalizacja wag
        #self.wagi = np.random.randn(liczba_neuronow, liczba_wejsc + 1) * 1/(np.sqrt(liczba_wejsc))
        self.wagi = np.random.normal(0,1/math.sqrt(liczba_wejsc),(1+liczba_wejsc)*liczba_neuronow)
        self.wagi = np.reshape(self.wagi, (liczba_neuronow, 1+liczba_wejsc )  )
        #self.wagi = np.random.randn(liczba_neuronow, liczba_wejsc + 1) * (2/(liczba_wejsc + liczba_neuronow))

        self.funkcja_aktywacji = funkcja_aktywacji
        self.wejscie = None
        self.net = None
        self.wyjscie = None
        
    def aktywacja(self, x):
        if self.funkcja_aktywacji == 'sigmoid':
            return 1 / (1 + np.exp(-np.clip(x, -250, 250)))
        elif self.funkcja_aktywacji == 'relu':
            return np.maximum(0, x)
        elif self.funkcja_aktywacji == 'tanh':
            return np.tanh(x)
        else:
            return x
    
    def pochodna_aktywacji(self, x):
        if self.funkcja_aktywacji == 'sigmoid':
            s = self.aktywacja(x)
            return s * (1 - s)
        elif self.funkcja_aktywacji == 'relu':
            return np.where(x > 0, 1, 0)
        elif self.funkcja_aktywacji == 'tanh':
            return 1 - np.tanh(x)**2
        else:
            return np.ones_like(x)
    
    def forward(self, X):
        # X może być pojedynczym wektorem (784,) lub batch (batch_size, 784)
        if X.ndim == 1:
            X = X.reshape(1, -1)
        
        # Dodajemy bias
        X_z_biasem = np.hstack([np.ones((X.shape[0], 1)), X])
        
        self.wejscie = X_z_biasem
        self.net = self.wejscie @ self.wagi.T
        self.wyjscie = self.aktywacja(self.net)
        
        return self.wyjscie
    
    def backward(self, delta_nastepna, stala_uczenia):
        delta = delta_nastepna * self.pochodna_aktywacji(self.net)
        
        # Oblicz gradienty wag
        dW = delta.T @ self.wejscie / self.wejscie.shape[0]
        
        # Aktualizuj wagi
        self.wagi -= stala_uczenia * dW
        
        # Oblicz błąd do przekazania do poprzedniej warstwy (bez biasu)
        delta_prev = delta @ self.wagi[:, 1:]
        
        return delta_prev

class SiecNeuronowa:
    def __init__(self, architektura, funkcje_aktywacji=None):
        self.warstwy = []
        
        if funkcje_aktywacji is None:
            # Domyślnie: relu dla warstw ukrytych, sigmoid dla wyjścia
            funkcje_aktywacji = ['relu'] * (len(architektura) - 2) + ['sigmoid']
        
        for i in range(len(architektura) - 1):
            warstwa = Warstwa(architektura[i], architektura[i+1], funkcje_aktywacji[i])
            self.warstwy.append(warstwa)
    
    def forward(self, X):
        wyjscie = X
        for warstwa in self.warstwy:
            wyjscie = warstwa.forward(wyjscie)
        return wyjscie
    
    def backward(self, y, stala_uczenia):
        # Oblicz błąd na wyjściu
        wyjscie_ost = self.warstwy[-1].wyjscie
        delta = wyjscie_ost - y
        
        # Propagacja wsteczna przez wszystkie warstwy
        for i in range(len(self.warstwy) - 1, -1, -1):
            delta = self.warstwy[i].backward(delta, stala_uczenia)
    
    def fit(self, training_data, epoki=10, stala_uczenia=0.1, rozmiar_batcha=32, verbose=True):
        """Trenowanie sieci na danych z MNIST loader"""
        historia_straty = []
        
        # Konwersja training_data z zip do listy
        training_data = list(training_data)
        
        for epoka in range(epoki):
            # Tasowanie danych
            np.random.shuffle(training_data)
            
            strata_epoki = 0
            liczba_batchy = 0
            
            for i in range(0, len(training_data), rozmiar_batcha):
                batch = training_data[i:i + rozmiar_batcha]
                
                # Przygotowanie batcha
                X_batch = np.array([x.reshape(-1) for x, y in batch])
                y_batch = np.array([y.reshape(-1) for x, y in batch])
                
                # Forward propagation
                wyjscie = self.forward(X_batch)
                
                # Oblicz stratę (mean squared error)
                strata = np.mean((wyjscie - y_batch) ** 2)
                strata_epoki += strata
                liczba_batchy += 1
                
                # Backward propagation
                self.backward(y_batch, stala_uczenia)
            
            strata_epoki /= liczba_batchy
            historia_straty.append(strata_epoki)
            
            if verbose:
                # Oblicz dokładność na zbiorze treningowym
                dokladnosc = self.accuracy(training_data[:1000])  # Tylko próbka dla szybkości
                print(f"Epoka {epoka}, Strata: {strata_epoki:.4f}, Dokładność: {dokladnosc:.4f}")
        
        return historia_straty
    
    def predict(self, X):
        wyjscie = self.forward(X)
        return np.argmax(wyjscie, axis=1)
    
    def accuracy(self, data):
        """Oblicza dokładność na danych oblicza dokladnosc na danych testiwych"""
        poprawne = 0
        total = 0
        
        for x, y in data:
            # Konwersja do formatu batch
            x_batch = x.reshape(1, -1)
            y_true = np.argmax(y)
            
            przewidywanie = self.predict(x_batch)[0]
            
            if przewidywanie == y_true:
                poprawne += 1
            total += 1
        
        return poprawne / total


# Prosty test na małym podzbiorze
def szybki_test():
    print("Szybki test...")
    training_data, validation_data, test_data = load_data_wrapper()
    training_data = list(training_data)
    test_data = list(test_data)

    #training_data = normuj_dane(training_data)
    #test_data = normuj_dane(test_data)
    
    # Mała sieć dla szybkiego testu
    siec = SiecNeuronowa([784, 128, 64, 10], ['sigmoid', 'sigmoid', 'sigmoid'])
    
    # Trenowanie na mniejszym zbiorze
    mini_training = training_data[:1000]
    siec.fit(mini_training, epoki=100, stala_uczenia=0.1, rozmiar_batcha=20)
    
    # Test na podzbiorze testowych
    dokladnosc = siec.accuracy(test_data[:100])
    print(f"Dokładność na {len(test_data[:100])} przykładach testowych: {dokladnosc:.4f}")

if __name__ == "__main__":
    szybki_test()  # Szybki test
    # eksperymentuj()  # Pełne eksperymenty

    # Stała uczenia 0.1 jest za czasami duża

Szybki test...
Epoka 0, Strata: 0.1055, Dokładność: 0.1170
Epoka 1, Strata: 0.0901, Dokładność: 0.1170
Epoka 2, Strata: 0.0899, Dokładność: 0.1180
Epoka 3, Strata: 0.0898, Dokładność: 0.1650
Epoka 4, Strata: 0.0898, Dokładność: 0.1850
Epoka 5, Strata: 0.0897, Dokładność: 0.1940
Epoka 6, Strata: 0.0897, Dokładność: 0.2110
Epoka 7, Strata: 0.0896, Dokładność: 0.1930
Epoka 8, Strata: 0.0896, Dokładność: 0.2040
Epoka 9, Strata: 0.0895, Dokładność: 0.2170
Epoka 10, Strata: 0.0895, Dokładność: 0.1340
Epoka 11, Strata: 0.0894, Dokładność: 0.2210
Epoka 12, Strata: 0.0894, Dokładność: 0.2050
Epoka 13, Strata: 0.0893, Dokładność: 0.1520
Epoka 14, Strata: 0.0892, Dokładność: 0.1200
Epoka 15, Strata: 0.0892, Dokładność: 0.2240
Epoka 16, Strata: 0.0891, Dokładność: 0.2230
Epoka 17, Strata: 0.0891, Dokładność: 0.2270
Epoka 18, Strata: 0.0890, Dokładność: 0.2560
Epoka 19, Strata: 0.0889, Dokładność: 0.2380
Epoka 20, Strata: 0.0889, Dokładność: 0.2250
Epoka 21, Strata: 0.0888, Dokładność: 0.2410
Epoka

In [ ]:
# Funkcja do eksperymentów
def eksperymentuj():
    print("Ładowanie danych MNIST...")
    training_data, validation_data, test_data = load_data_wrapper()
    
    # Konwersja do list (ważne dla Python 3)
    training_data = list(training_data)
    validation_data = list(validation_data)
    test_data = list(test_data)
    
    print(f"Liczba przykładów treningowych: {len(training_data)}")
    print(f"Liczba przykładów testowych: {len(test_data)}")
    
    # Różne architektury do przetestowania
    architektury = [
        [784, 128, 10],           # 1 warstwa ukryta, 128 neuronów
        [784, 256, 10],           # 1 warstwa ukryta, 256 neuronów  
        [784, 64, 64, 10],        # 2 warstwy ukryte, 64 neurony każda
        [784, 128, 64, 10],       # 2 warstwy ukryte, różne rozmiary
    ]
    
    for i, architektura in enumerate(architektury):
        print(f"\n{'='*50}")
        print(f"Eksperyment {i+1}: {architektura}")
        print(f"{'='*50}")
        
        # ReLU dla warstw ukrytych, sigmoid dla wyjścia
        funkcje_aktywacji = ['relu'] * (len(architektura) - 2) + ['sigmoid']
        
        siec = SiecNeuronowa(architektura, funkcje_aktywacji)
        
        print("Trenowanie...")
        historia = siec.fit(training_data, epoki=20, stala_uczenia=0.1, rozmiar_batcha=64)
        
        # Testowanie na zbiorze testowym
        dokladnosc_test = siec.accuracy(test_data[:1000])  # Tylko próbka dla szybkości
        dokladnosc_trening = siec.accuracy(training_data[:1000])
        
        print(f"\nWyniki:")
        print(f"Dokładność na zbiorze treningowym: {dokladnosc_trening:.4f}")
        print(f"Dokładność na zbiorze testowym: {dokladnosc_test:.4f}")

In [ ]:
import numpy as np
from mnist_loader import load_data_wrapper

class Warstwa:
    def __init__(self, liczba_wejsc, liczba_neuronow, funkcja_aktywacji='sigmoid'):
        self.liczba_wejsc = liczba_wejsc
        self.liczba_neuronow = liczba_neuronow
        self.wagi = np.random.randn(liczba_neuronow, liczba_wejsc + 1) * np.sqrt(2.0 / (liczba_wejsc + liczba_neuronow))
        self.funkcja_aktywacji = funkcja_aktywacji
        self.wejscie = None
        self.net = None
        self.wyjscie = None
        
    def aktywacja(self, x):
        if self.funkcja_aktywacji == 'sigmoid':
            x = np.clip(x, -20, 20)
            return 1 / (1 + np.exp(-x))
        elif self.funkcja_aktywacji == 'relu':
            return np.maximum(0.01 * x, x)
        elif self.funkcja_aktywacji == 'tanh':
            return np.tanh(x)
        elif self.funkcja_aktywacji == 'softmax':
            exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
            return exp_x / np.sum(exp_x, axis=1, keepdims=True)
        else:
            return x
    
    def pochodna_aktywacji(self, x):
        if self.funkcja_aktywacji == 'sigmoid':
            s = self.aktywacja(x)
            return s * (1 - s)
        elif self.funkcja_aktywacji == 'relu':
            return np.where(x > 0, 1, 0.01)
        elif self.funkcja_aktywacji == 'tanh':
            return 1 - np.tanh(x)**2
        elif self.funkcja_aktywacji == 'softmax':
            return 1.0
        else:
            return np.ones_like(x)
    
    def forward(self, X):
        if X.ndim == 1:
            X = X.reshape(1, -1)
        
        X_z_biasem = np.hstack([np.ones((X.shape[0], 1)), X])
        self.wejscie = X_z_biasem
        self.net = self.wejscie @ self.wagi.T
        self.wyjscie = self.aktywacja(self.net)
        return self.wyjscie
    
    def backward(self, delta_nastepna, stala_uczenia):
        delta_nastepna = np.clip(delta_nastepna, -1, 1)
        delta = delta_nastepna * self.pochodna_aktywacji(self.net)
        delta = np.clip(delta, -1, 1)
        
        dW = delta.T @ self.wejscie / self.wejscie.shape[0]
        dW = np.clip(dW, -0.1, 0.1)
        
        self.wagi -= stala_uczenia * dW
        delta_prev = delta @ self.wagi[:, 1:]
        return delta_prev

class SiecNeuronowa:
    def __init__(self, architektura, funkcje_aktywacji=None):
        self.warstwy = []
        
        if funkcje_aktywacji is None:
            funkcje_aktywacji = ['relu'] * (len(architektura) - 2) + ['softmax']
        
        if len(funkcje_aktywacji) != len(architektura) - 1:
            required = len(architektura) - 1
            given = len(funkcje_aktywacji)
            raise ValueError(f"Błąd: Potrzebujesz {required} funkcji aktywacji, ale podano {given}")
        
        for i in range(len(architektura) - 1):
            warstwa = Warstwa(architektura[i], architektura[i+1], funkcje_aktywacji[i])
            self.warstwy.append(warstwa)
    
    def forward(self, X):
        wyjscie = X
        for warstwa in self.warstwy:
            wyjscie = warstwa.forward(wyjscie)
        return wyjscie
    
    def backward(self, y, stala_uczenia):
        wyjscie_ost = self.warstwy[-1].wyjscie
        
        if self.warstwy[-1].funkcja_aktywacji == 'softmax':
            delta = wyjscie_ost - y
        else:
            delta = wyjscie_ost - y
            
        delta = np.clip(delta, -1, 1)
        
        for i in range(len(self.warstwy) - 1, -1, -1):
            delta = self.warstwy[i].backward(delta, stala_uczenia)
    
    def compute_loss(self, wyjscie, y):
        epsilon = 1e-8
        wyjscie = np.clip(wyjscie, epsilon, 1 - epsilon)
        return -np.mean(np.sum(y * np.log(wyjscie), axis=1))
    
    def fit(self, training_data, validation_data=None, epoki=10, stala_uczenia=0.01, rozmiar_batcha=32, verbose=True):
        """Dodajemy validation_data jako opcjonalny parametr"""
        historia_straty = []
        training_data = list(training_data)
        
        for epoka in range(epoki):
            np.random.shuffle(training_data)
            
            strata_epoki = 0
            liczba_batchy = 0
            
            for i in range(0, len(training_data), rozmiar_batcha):
                batch = training_data[i:i + rozmiar_batcha]
                
                X_batch = np.array([(x / 255.0).reshape(-1) for x, y in batch])
                y_batch = np.array([y.reshape(-1) for x, y in batch])
                
                wyjscie = self.forward(X_batch)
                
                if self.warstwy[-1].funkcja_aktywacji == 'softmax':
                    strata = self.compute_loss(wyjscie, y_batch)
                else:
                    strata = np.mean((wyjscie - y_batch) ** 2)
                    
                if np.isnan(strata):
                    continue
                    
                strata_epoki += strata
                liczba_batchy += 1
                
                self.backward(y_batch, stala_uczenia)
            
            if liczba_batchy > 0:
                strata_epoki /= liczba_batchy
                historia_straty.append(strata_epoki)
            
            if verbose and epoka % 5 == 0:
                dokladnosc_trening = self.accuracy(training_data[:500])
                
                # Sprawdzaj validation_data jeśli jest podane
                if validation_data is not None:
                    dokladnosc_val = self.accuracy(validation_data[:200])
                    print(f"Epoka {epoka}, Strata: {strata_epoki:.4f}, Train: {dokladnosc_trening:.4f}, Val: {dokladnosc_val:.4f}")
                else:
                    print(f"Epoka {epoka}, Strata: {strata_epoki:.4f}, Train Acc: {dokladnosc_trening:.4f}")
        
        return historia_straty
    
    def predict(self, X):
        if isinstance(X, np.ndarray):
            if np.max(X) > 1.0:
                X = X / 255.0
        wyjscie = self.forward(X)
        return np.argmax(wyjscie, axis=1)
    
    def accuracy(self, data):
        poprawne = 0
        total = 0
        
        for x, y in data:
            x_norm = (x / 255.0).reshape(1, -1)
            y_true = np.argmax(y)
            
            przewidywanie = self.predict(x_norm)[0]
            
            if przewidywanie == y_true:
                poprawne += 1
            total += 1
        
        return poprawne / total

def szybki_test():
    print("Szybki test...")
    training_data, validation_data, test_data = load_data_wrapper()
    training_data = list(training_data)
    validation_data = list(validation_data)
    test_data = list(test_data)
    
    # Użyj softmax dla lepszej klasyfikacji
    siec = SiecNeuronowa([784, 128, 64, 10], ['relu', 'relu', 'softmax'])
    
    # Trenuj na mniejszym zbiorze z walidacją
    mini_training = training_data[:5000]
    mini_validation = validation_data[:1000]  # Użyj zbioru walidacyjnego
    
    print("Trenowanie...")
    # Przekaż validation_data do monitorowania overfittingu
    siec.fit(mini_training, validation_data=mini_validation, epoki=30, 
             stala_uczenia=0.01, rozmiar_batcha=64, verbose=True)
    
    print("\nFinalne testowanie...")
    dokladnosc_test = siec.accuracy(test_data[:5000])
    dokladnosc_trening = siec.accuracy(mini_training[:1000])
    
    print(f"Dokładność na zbiorze treningowym: {dokladnosc_trening:.4f}")
    print(f"Dokładność na zbiorze testowym: {dokladnosc_test:.4f}")

if __name__ == "__main__":
    szybki_test()

Szybki test...
Trenowanie...
Epoka 0, Strata: 2.3283, Train: 0.1020, Val: 0.0000
Epoka 5, Strata: 2.3041, Train: 0.1040, Val: 0.0000
Epoka 10, Strata: 2.3004, Train: 0.0860, Val: 0.0000
Epoka 15, Strata: 2.2997, Train: 0.1020, Val: 0.0000
Epoka 20, Strata: 2.3000, Train: 0.1260, Val: 0.0000
Epoka 25, Strata: 2.2991, Train: 0.1080, Val: 0.0000

Finalne testowanie...
Dokładność na zbiorze treningowym: 0.1160
Dokładność na zbiorze testowym: 0.0000
